In [0]:
SELECT 
*,
EXPLODE(o.books) as book

   FROM orders_silver AS o
   where o.order_id = '000000004766'
   ;
  
SELECT 
order_id,
customer_id,
book,
book.*
   FROM orders_silver AS o,
        LATERAL EXPLODE(o.books) AS b(book)
   where o.order_id = '000000004766'
   ;

select * from books_sales
where author = 'Tracy Kidder'
order by order_timestamp
;

select * from authors_stats
where author = 'Tracy Kidder'
order by window_start
;

-- ##### window function
create temporary view test_window
as(
select  date_format(stamp, 'hh:mm:ss') as stamp,  val
  FROM VALUES (TIMESTAMP'2020-08-01 12:20:21', 17),
              (TIMESTAMP'2020-08-01 12:20:22', 12),
              (TIMESTAMP'2020-08-01 12:23:10',  8),
              (TIMESTAMP'2020-08-01 12:25:05', 11),
              (TIMESTAMP'2020-08-01 12:28:59', 15),
              (TIMESTAMP'2020-08-01 12:30:01', 23),
              (TIMESTAMP'2020-08-01 12:30:15',  2),
              (TIMESTAMP'2020-08-01 12:35:22', 16) 
AS S(stamp, val)
);
select * from test_window
;

SELECT  
date_format(window.start, 'hh:mm:ss'),
date_format(window.end,'hh:mm:ss'),
window.end - window.start as duration,
date_format(window.start + duration,'hh:mm:ss') as end_plus_duration,
min(val), max(val), count(val)
from test_window
  GROUP BY window(stamp, '2 MINUTES 30 SECONDS', '30 SECONDS', '15 SECONDS')
;


SELECT
  author,
  window.start AS window_start,
  window.end AS window_end,
  COUNT("order_id") AS orders_count,
  AVG("quantity") AS avg_quantity
FROM books_sales
GROUP BY
  author,
  window(order_timestamp, '5 minutes', '5 minutes', '2 minutes')
ORDER BY
  window_start;
